In [1]:
from IPython.display import display, HTML
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

import numpy as np
import warnings


from datetime import datetime
import glob
import io, os , sys, types
import tabulate
import copy

import random

import matplotlib.pyplot as plt
%matplotlib inline
import itertools

from sklearn.model_selection import train_test_split
from sklearn import linear_model
#from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB 
from sklearn import datasets
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import tree
from sklearn.externals.six import StringIO
from sklearn.ensemble import RandomForestClassifier
import pydotplus

from scipy import stats
from scipy.stats import pearsonr
from scipy.stats import chi2_contingency

from helper_functions import *

import seaborn as sns
sns.set(color_codes=True)

fontsz = 12

# ROC Curve and Cutoff Analysis:
# https://ncss-wpengine.netdna-ssl.com/wp-content/themes/ncss/pdf/Procedures/NCSS/One_ROC_Curve_and_Cutoff_Analysis.pdf

In [2]:
# load the dataset
fname_germancredit = r'dataset/German.Credit.csv'
data_raw = pd.read_csv(fname_germancredit)

In [3]:
data_raw.describe()

,duration,credit_amount,installment_commitment,residence_since,age,existing_credits,num_dependents
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,20.903000,3271.258000,2.973000,2.845000,35.546000,1.407000,1.155000
std,12.058814,2822.736876,1.118715,1.103718,11.375469,0.577654,0.362086
min,4.000000,250.000000,1.000000,1.000000,19.000000,1.000000,1.000000
25%,12.000000,1365.500000,2.000000,2.000000,27.000000,1.000000,1.000000
50%,18.000000,2319.500000,3.000000,3.000000,33.000000,1.000000,1.000000
75%,24.000000,3972.250000,4.000000,4.000000,42.000000,2.000000,1.000000
max,72.000000,18424.000000,4.000000,4.000000,75.000000,4.000000,2.000000


In [4]:
col_target = 'class'
cols_numeric = list(data_raw.describe().columns.values)
cols_categoric = list(set(data_raw.columns.values) - set(cols_numeric) - set([col_target]))

In [5]:
cols_categoric

['property_magnitude',
 'own_telephone',
 'savings_status',
 'credit_history',
 'personal_status',
 'employment',
 'other_parties',
 'other_payment_plans',
 'purpose',
 'housing',
 'account_balance',
 'foreign',
 'job']

In [6]:
# Contingency table
pd.crosstab(data_raw['class'], data_raw['account_balance'],  margins=True)

account_balance,'0<=X<200','<0','>=200','no account',All
class,,,,,
bad,105,135,14,46,300
good,164,139,49,348,700
All,269,274,63,394,1000


In [7]:
# Contingency table, ratios. Rows add-up to 100%
pd.crosstab(data_raw['class'], data_raw['account_balance'],  margins=False, normalize='index')

account_balance,'0<=X<200','<0','>=200','no account'
class,,,,
bad,0.350000,0.450000,0.046667,0.153333
good,0.234286,0.198571,0.070000,0.497143


In [8]:
# Contingency table, ratios. Columns add-up to 100%
pd.crosstab(data_raw['class'], data_raw['account_balance'],  margins=False, normalize='columns')

account_balance,'0<=X<200','<0','>=200','no account'
class,,,,
bad,0.390335,0.492701,0.222222,0.116751
good,0.609665,0.507299,0.777778,0.883249


In [9]:
contingency = pd.crosstab(data_raw['class'], data_raw['account_balance'])
c, pval, dof, expected = chi2_contingency(contingency)
print("p-value:",pval, '\t variable:', 'account_balance')

p-value: 1.21890207229e-26 	 variable: account_balance


In [10]:
for col in cols_categoric:
    contingency = pd.crosstab(data_raw['class'], data_raw[col])
    c, pval, dof, expected = chi2_contingency(contingency)
    print("p-value:",pval, '\t variable:', col)

p-value: 2.85844157333e-05 	 variable: property_magnitude
p-value: 0.278876154304 	 variable: own_telephone
p-value: 2.76121423857e-07 	 variable: savings_status
p-value: 1.27918729568e-12 	 variable: credit_history
p-value: 0.0222380054693 	 variable: personal_status
p-value: 0.00104545234914 	 variable: employment
p-value: 0.0360559540272 	 variable: other_parties
p-value: 0.00162931781865 	 variable: other_payment_plans
p-value: 0.000115749100797 	 variable: purpose
p-value: 0.000111674653746 	 variable: housing
p-value: 1.21890207229e-26 	 variable: account_balance
p-value: 0.0158307549029 	 variable: foreign
p-value: 0.596581591884 	 variable: job


#### Descriptive Statistics for Numerical Predictors

In [ ]:
# Correlation between numeric variables
data_numeric = data_raw[cols_numeric].copy(deep=True)
corr_mat = data_numeric.corr(method='pearson')
cbar_ticks =np.linspace(-1,1,11)
cmap = sns.diverging_palette(220, 10, as_cmap=True)
plt.figure(figsize=[8,8])
plt.xticks(fontsize=fontsz+2)
plt.yticks(fontsize=fontsz+2)
ax = sns.heatmap(corr_mat, cmap=cmap, vmin=-1, vmax=1, square=True, linewidths=.5, cbar_kws={"shrink": .5})
cbar = ax.collections[0].colorbar
cbar.set_ticks(cbar_ticks)
cbar.set_ticklabels(cbar_ticks)
plt.show()

In [ ]:
print (corr_mat)

#### Histograms 

In [ ]:
brksCredits = np.linspace(0,80,11) # Bins for a nice looking histogram
plt.figure(figsize=(10,5))
plt.hist(data_raw['duration'], bins=brksCredits)
plt.title('duration', fontsize=fontsz+4)
plt.xlabel('Loan Period [Months]', fontsize=fontsz+2)
plt.ylabel('Count', fontsize=fontsz+2)
plt.xticks(fontsize=fontsz)
plt.yticks(fontsize=fontsz)
plt.show()

In [ ]:
plt.figure(figsize=(5,5))
plt.boxplot(data_raw['duration']) 
plt.title('duration boxplot', fontsize=fontsz+4)
plt.xlabel('Credit Month', fontsize=fontsz+2) 
plt.ylabel('duration', fontsize=fontsz+2)
plt.xticks(fontsize=fontsz)
plt.yticks(fontsize=fontsz)
plt.show()

#### Preprocessing
Dummy variables

In [ ]:
print ("Number of columns before dummy-variables:\t", len(data_raw.columns.values))
for i in cols_categoric:
    dummy_ranks = pd.get_dummies(data_raw[i], prefix=i)
    data_raw = data_raw.join(dummy_ranks)
    # dropping the original categoric column (not needed - it was replaced by dummy columns)
    data_raw = data_raw.drop(i, 1) 
    
# all feature, numeric, and categoric (now dummified)
cols_features = list(set(data_raw.columns.values) - set([col_target])) 
print ("Number of columns after dummy-variables:\t", len(data_raw.columns.values))

In [ ]:
# Replacing calss variable with numeric values (for convinience)
data_raw['class'].replace('bad', 0, inplace=True)
data_raw['class'].replace('good', 1, inplace=True)
data_raw['class'] = pd.to_numeric(data_raw['class'])

In [ ]:
# Random seed
seed = 1017
random.seed(seed)
frac_train = 0.8 # 80% of the data is used for training
X_train, X_test, y_train, y_test = \
    train_test_split(data_raw[cols_features], data_raw[col_target], test_size=(1-frac_train), random_state=seed)
    
train_b = sum(y_train == 0)
train_g = sum(y_train == 1)
test_b = sum(y_test == 0)
test_g = sum(y_test == 1)
print ("Class ratios between each set:")
print ("Trainset")
print ("\t\tNormal class (good):", 100*train_g/len(y_train), "%\t", "Target class (bad):", 100*train_b/len(y_train),"%")
print ("Testset")
print ("\t\tNormal class (good):", 100*test_g/len(y_test), "%\t", "Target class (bad):", 100*test_b/len(y_test),"%")

In [ ]:
# Set Misclassification loss weights
c_tn = 0 # weight of true-negative
c_tp = 0 # weight of true-positive
c_fn = 1 # weight of false negative
c_fp = 5 # weight of false positive

### Logistic Regression Model
More about Logistic Regression examples in python can be found here:<br>
http://blog.yhat.com/posts/logistic-regression-python-rodeo.html

In [ ]:
model = linear_model.LogisticRegression()
model.fit(X_train, y_train)

In [ ]:
mse = np.mean(y_train - model.predict(X_train)) ** 2
print ("Mean Square Error: ", mse)

In [ ]:
model_coefficients = model.coef_[0]
df_lgm_coeffs = pd.DataFrame(data=model_coefficients, index=cols_features, columns=['LGM_coeff'])
# sort by coefficients absolute value
df_lgm_coeffs = df_lgm_coeffs.reindex(df_lgm_coeffs.LGM_coeff.abs().sort_values(ascending=False).index)
display(df_lgm_coeffs)

In [ ]:
predicted = model.predict(X_test)
predicted_prob = model.predict_proba(X_test)[:,1]

In [ ]:
fpr, tpr, _ = metrics.roc_curve(np.array(y_test), predicted_prob)
auc = metrics.auc(fpr,tpr)
print ("Area-Under-Curve:", round(auc,4))
# plot_ROC() is defined in helper_functions.py
plot_ROC(fpr,tpr, fontsz, 'Receiver operating characteristic for Logistic Regression Model') 

In [ ]:
# finding the optimal values using the TRAIN-SET
train_predicted_prob = model.predict_proba(X_train)[:,1]
loss_matrix = calculate_loss(train_predicted_prob, y_train, c_fn, c_fp, c_tp, c_tn) 
# finding optimal threshold:
opt_thr = list(loss_matrix[loss_matrix['loss'] == loss_matrix['loss'].min()]['prediction'])[0]
print("Optimal threshold at:\t",round(opt_thr,5))
print("Model Loss:", loss_matrix['loss'].min())
loss = loss_matrix['loss'].min()
predicted_prob_opt = copy.deepcopy(predicted_prob)
predicted_prob_opt[predicted_prob_opt >  opt_thr] = 1
predicted_prob_opt[predicted_prob_opt <= opt_thr] = 0

In [ ]:
def_cfm = metrics.confusion_matrix(y_test, predicted) # default confusion matrix, default threshold = 0.5
opt_cfm = metrics.confusion_matrix(y_test, predicted_prob_opt) # optimal threshold

In [ ]:
plot_confusion_matrix(def_cfm,['bad', 'good'], "Default Confusion Matrix", 0)
plot_confusion_matrix(opt_cfm,['bad', 'good'], "Loss-Optimized Confusion Matrix", 1)
plt.show()

In [ ]:
plt.figure(figsize=(10,5), facecolor='white')
plt.plot(loss_matrix['prediction'], loss_matrix['loss'], 'o-')
plt.title('Loss function for various thresholds and confusion matrices', fontsize=fontsz+4)
plt.xlabel('Threshold', fontsize=fontsz+4)
plt.ylabel('Loss', fontsize=fontsz+4)
plt.show()

## Naive Bayes Model

In [ ]:
gnb = BernoulliNB()
model = gnb.fit(X_train, y_train)
predicted = model.predict(X_test)
predicted_prob = model.predict_proba(X_test)[:, 1]

In [ ]:
model_coefficients = model.coef_[0]
df_coeffs = pd.DataFrame(data=[list(cols_features), list(model_coefficients)]).transpose()
df_coeffs.columns = ['feature', 'coeff']
# sort by coefficients absolute value (log probability of the positive class)
df_coeffs = df_coeffs.reindex(df_coeffs['coeff'].abs().sort_values(inplace=False, ascending=True).index)
display(df_coeffs)

In [ ]:
fpr, tpr, _ = metrics.roc_curve(np.array(y_test), predicted_prob)
auc = metrics.auc(fpr,tpr)
print ("Area-Under-Curve:", round(auc,4))
plot_ROC(fpr,tpr, fontsz, 'Receiver operating characteristic for Naive Bayes Model') 

In [ ]:
# finding the optimal values using the TRAIN-SET
train_predicted_prob = model.predict_proba(X_train)[:,1]
loss_matrix = calculate_loss(train_predicted_prob, y_train, c_fn, c_fp, c_tp, c_tn) 
# finding optimal threshold:
opt_thr = list(loss_matrix[loss_matrix['loss'] == loss_matrix['loss'].min()]['prediction'])[0]
print("Optimal threshold at:\t",round(opt_thr,5))
loss = loss_matrix['loss'].min()
print("Model Loss:", loss)
predicted_prob_opt = copy.deepcopy(predicted_prob)
predicted_prob_opt[predicted_prob_opt >  opt_thr] = 1
predicted_prob_opt[predicted_prob_opt <= opt_thr] = 0

In [ ]:
def_cfm = metrics.confusion_matrix(y_test, predicted) 
opt_cfm = metrics.confusion_matrix(y_test, predicted_prob_opt) # optimal threshold

In [ ]:
plot_confusion_matrix(def_cfm,['bad', 'good'], "Default Confusion Matrix", 0)
plot_confusion_matrix(opt_cfm,['bad', 'good'], "Loss-Optimized Confusion Matrix", 1)
plt.show()

## Decision Trees
Decision Trees is a recursive-repartitioning technique, which is used to recursively split the data in order to create nodes that are<br>
purer. A pure node is a node that consists of only 1-class of those existing in the data.<br>
In our context, a pure node would be composed of either all-"bad" or all-"good" classes.<br>
The advantages of DT is that it produces rules that are easy to follow, and human-readable, in contrast to other "black-box" algorithms, such as Random-Forest<br>
DTs however, are prone to overfitting, which is why we need to use some parameters to avoid such behavior.<br>
As with __Logistic Regression__, __DT__s also require categorical features to be dummified.<br>
1. Based on what we discussed, can you offer an intuition about why DTs tend to overfit?
2. [Advanced] Can you offer some ways to avoid overfitting?

In [ ]:
md = 18                    # maximum tree depth
mf = len(cols_features)    # maximum number of features to consider
min_leaf = 10
criterion = 'entropy'
model = tree.DecisionTreeClassifier(max_depth=md, max_features=mf, criterion=criterion, 
                                    min_samples_leaf=min_leaf, random_state=seed)

clf = model.fit(X_train, y_train)
predicted = model.predict(X_test)
predicted_prob = model.predict_proba(X_test)[:, 1]

In [ ]:
importance = model.feature_importances_
df_importance = pd.DataFrame(data=[list(cols_features), list(importance)]).transpose()
df_importance.columns = ['feature', 'importance']
df_importance = df_importance[df_importance['importance'] != 0]
# sort by feature importance
df_importance = df_importance.reindex(df_importance['importance'].abs().sort_values(inplace=False, ascending=False).index)
display(df_importance)

In [ ]:
# Visualize the tree
#write_Tree('dataset', clf, cols_features) # can be used only for small trees (<=4)

In [ ]:
fpr, tpr, _ = metrics.roc_curve(np.array(y_test), predicted_prob)
auc = metrics.auc(fpr,tpr)
print ("Area-Under-Curve:", round(auc,4))
plot_ROC(fpr,tpr, fontsz, "Receiver operating characteristic for Decision Tree Model") 

In [ ]:
# finding the optimal values using the TRAIN-SET
train_predicted_prob = model.predict_proba(X_train)[:,1]
loss_matrix = calculate_loss(train_predicted_prob, y_train, c_fn, c_fp, c_tp, c_tn) 
# finding optimal threshold:
opt_thr = list(loss_matrix[loss_matrix['loss'] == loss_matrix['loss'].min()]['prediction'])[0]
print("Optimal threshold at:\t",round(opt_thr,5))
loss = loss_matrix['loss'].min()
print("Model Loss:", loss)
predicted_prob_opt = copy.deepcopy(predicted_prob)
predicted_prob_opt[predicted_prob_opt >  opt_thr] = 1
predicted_prob_opt[predicted_prob_opt <= opt_thr] = 0

In [ ]:
def_cfm = metrics.confusion_matrix(y_test, predicted) 
opt_cfm = metrics.confusion_matrix(y_test, predicted_prob_opt) # optimal threshold

In [ ]:
plot_confusion_matrix(def_cfm,['bad', 'good'], "Default Confusion Matrix", 0)
plot_confusion_matrix(opt_cfm,['bad', 'good'], "Loss-Optimized Confusion Matrix", 1)
plt.show()

## Random Forest
Random Forest is an ensemble learning classification method, which utilizes multiple decision-trees,<br>
and a voting mechanism in order to classify each sample.

In [ ]:
model = RandomForestClassifier(max_depth=md, max_features=mf, 
                               criterion=criterion, min_samples_leaf = min_leaf, random_state=seed)
clf = model.fit(X_train, y_train)

In [ ]:
importance = model.feature_importances_
df_importance = pd.DataFrame(data=[list(cols_features), list(importance)]).transpose()
df_importance.columns = ['feature', 'importance']
df_importance = df_importance[df_importance['importance'] != 0]
# sort by feature importance
df_importance = df_importance.reindex(df_importance['importance'].abs().sort_values(inplace=False, ascending=False).index)
display(df_importance)

In [ ]:
predicted = clf.predict(X_test)
predicted_prob = clf.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = metrics.roc_curve(np.array(y_test), predicted_prob)
auc = metrics.auc(fpr,tpr)
print ("Area-Under-Curve:", round(auc,4))
plot_ROC(fpr,tpr, fontsz, "Receiver operating characteristic for Random Forest Model")

In [ ]:
# finding the optimal values using the TRAIN-SET
train_predicted_prob = model.predict_proba(X_train)[:,1]
loss_matrix = calculate_loss(train_predicted_prob, y_train, c_fn, c_fp, c_tp, c_tn) 
# finding optimal threshold:
opt_thr = list(loss_matrix[loss_matrix['loss'] == loss_matrix['loss'].min()]['prediction'])[0]
print("Optimal threshold at:\t",round(opt_thr,5))
loss = loss_matrix['loss'].min()
print("Model Loss:", loss)
predicted_prob_opt = copy.deepcopy(predicted_prob)
predicted_prob_opt[predicted_prob_opt >  opt_thr] = 1
predicted_prob_opt[predicted_prob_opt <= opt_thr] = 0

In [ ]:
def_cfm = metrics.confusion_matrix(y_test, predicted) 
opt_cfm = metrics.confusion_matrix(y_test, predicted_prob_opt) # optimal threshold

In [ ]:
plot_confusion_matrix(def_cfm,['bad', 'good'], "Default Confusion Matrix", 0)
plot_confusion_matrix(opt_cfm,['bad', 'good'], "Loss-Optimized Confusion Matrix", 1)
plt.show()

In [ ]:
# model AUC, loss:
# Logistic Regression:    0.81, 354
# Naive Bayes:            0.81, 395
# Decision Tree:          0.71, 271
# Random Forest:          0.77, 241 